In [ ]:
import sys
from tensorflow import keras as k
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from matplotlib import pyplot
from tqdm.keras import TqdmCallback

In [ ]:
##
#Basic implementation (see He (2015) p.7)
##
#output size 32*32 16*16 8*8
#layers      1+2n  2n    2n
#filters     16    32    64
#We use n=3 for now

def define_resnet(wdecay):
    inputs = k.Input(shape=(32, 32, 3))
    layer1 = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(inputs)
    layer1 = k.layers.BatchNormalization()(layer1)
    
    #FIRST STACK
    x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(layer1)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    #Adding skip connection to output of residual block
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([layer1, x]))
    x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    #SECOND STACK
    #Subsampling with stride 2
    x = k.layers.Conv2D(32, (3, 3), strides=(2,2), activation='relu', kernel_initializer='he_uniform',
        padding='same', kernel_regularizer=k.regularizers.l2(wdecay))(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Conv2D(32, (1, 1), strides=(2,2), kernel_initializer='he_uniform', padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    #THIRD STACK
    x = k.layers.Conv2D(64, (3, 3), strides=(2,2), activation='relu', kernel_initializer='he_uniform',
        padding='same', kernel_regularizer=k.regularizers.l2(wdecay))(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Conv2D(64, (1, 1), strides=(2,2), kernel_initializer='he_uniform', padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
        kernel_regularizer=k.regularizers.l2(wdecay))(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    outputs = k.layers.Dense(10, activation='softmax')(k.layers.Flatten()(res))

    return k.Model(inputs=inputs, outputs=outputs, name='ResNet_20')

In [ ]:
resnet = define_resnet(0)
resnet.summary()

In [ ]:
def summarize_diagnostics(history):
 # plot loss
 pyplot.subplot(211)
 pyplot.title('Cross Entropy Loss')
 pyplot.plot(history.history['loss'], color='blue', label='train')
 pyplot.plot(history.history['val_loss'], color='orange', label='test')
 # plot accuracy
 pyplot.subplot(212)
 pyplot.title('Classification Accuracy')
 pyplot.plot(history.history['accuracy'], color='blue', label='train')
 pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
 # save plot to file
 filename = sys.argv[0].split('/')[-1]
 pyplot.tight_layout()
 pyplot.savefig(filename + '_plot.png')
 pyplot.close()

In [ ]:
def run_test(ep=180, learning_rate=0.1, wdecay=0):
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels_mean(trainX, testX)
    # define model
    model = define_resnet(wdecay)
    opt = k.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(optimizer=opt, loss=k.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    lr_sched = k.callbacks.LearningRateScheduler(resnet_scheduler)
    history = model.fit(trainX, trainY, epochs=ep, batch_size=128, validation_data=(testX, testY), verbose=0, callbacks=TqdmCallback(verbose=1))
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

In [ ]:
run_test()